In [1]:
# Importing Packages
import os
from PIL import Image
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
import math
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from skimage import transform
from sys import getsizeof
from functions_cnn import *

# Loading Images
newImageResolution = 100
FLD1  = makeCubeArray('Scans/FLD_1_cut.mhd', newImageResolution, adjustScans=True)
FLD2  = makeCubeArray('Scans/FLD_2_cut.mhd', newImageResolution, adjustScans=True)
FLD3  = makeCubeArray('Scans/FLD_3_cut.mhd', newImageResolution, adjustScans=True)
FLD10 = makeCubeArray('Scans/FLD_10_cut.mhd', newImageResolution, adjustScans=True)
FLD11 = makeCubeArray('Scans/FLD_11_cut.mhd', newImageResolution, adjustScans=True)
FLD12 = makeCubeArray('Scans/FLD_12_cut.mhd', newImageResolution, adjustScans=True)
F1 = makeCubeArray('Scans/041022_Blarr_CF_20-47_FLD_C-F_3_F_1-3-0123,00_recon_F1_BC.mhd', newImageResolution, adjustScans=True)
F2 = makeCubeArray('Scans/041022_Blarr_CF_20-47_FLD_C-F_3_F_1-3-0123,00_recon_F2_BC.mhd', newImageResolution, adjustScans=True)
F3 = makeCubeArray('Scans/041022_Blarr_CF_20-47_FLD_C-F_3_F_1-3-0123,00_recon_F3_BC.mhd', newImageResolution, adjustScans=True)
CF1 = makeCubeArray('Scans/041022_Blarr_CF_20-47_FLD_C-F_3_F_1-3-0123,00_recon_CF1_BC.mhd', newImageResolution, adjustScans=True)
CF2 = makeCubeArray('Scans/041022_Blarr_CF_20-47_FLD_C-F_3_F_1-3-0123,00_recon_CF2_BC.mhd', newImageResolution, adjustScans=True)
CF3 = makeCubeArray('Scans/041022_Blarr_CF_20-47_FLD_C-F_3_F_1-3-0123,00_recon_CF3_BC.mhd', newImageResolution, adjustScans=True)
C1 = makeCubeArray('Scans/041022_Blarr_CF_20-47_FLD_C-F_3_F_1-3-0123,00_recon_C1_BC.mhd', newImageResolution, adjustScans=True)
C2 = makeCubeArray('Scans/041022_Blarr_CF_20-47_FLD_C-F_3_F_1-3-0123,00_recon_C2_BC.mhd', newImageResolution, adjustScans=True)
C3 = makeCubeArray('Scans/041022_Blarr_CF_20-47_FLD_C-F_3_F_1-3-0123,00_recon_C3_BC.mhd', newImageResolution, adjustScans=True)

Scans/FLD_1_cut.mhd loaded (100, 100, 100) 

Scans/FLD_2_cut.mhd loaded (100, 100, 100) 

Scans/FLD_3_cut.mhd loaded (100, 100, 100) 

Scans/FLD_10_cut.mhd loaded (100, 100, 100) 

Scans/FLD_11_cut.mhd loaded (100, 100, 100) 

Scans/FLD_12_cut.mhd loaded (100, 100, 100) 

Scans/TGA_1_cut.mhd loaded (100, 100, 100) 

Scans/TGA_4_cut.mhd loaded (100, 100, 100) 

Scans/041022_Blarr_CF_20-47_FLD_C-F_3_F_1-3-0123,00_recon_F1_BC.mhd loaded (100, 100, 100) 

Scans/041022_Blarr_CF_20-47_FLD_C-F_3_F_1-3-0123,00_recon_F2_BC.mhd loaded (100, 100, 100) 

Scans/041022_Blarr_CF_20-47_FLD_C-F_3_F_1-3-0123,00_recon_F3_BC.mhd loaded (100, 100, 100) 

Scans/041022_Blarr_CF_20-47_FLD_C-F_3_F_1-3-0123,00_recon_CF1_BC.mhd loaded (100, 100, 100) 

Scans/041022_Blarr_CF_20-47_FLD_C-F_3_F_1-3-0123,00_recon_CF2_BC.mhd loaded (100, 100, 100) 

Scans/041022_Blarr_CF_20-47_FLD_C-F_3_F_1-3-0123,00_recon_CF3_BC.mhd loaded (100, 100, 100) 

Scans/041022_Blarr_CF_20-47_FLD_C-F_3_F_1-3-0123,00_recon_C1_BC.mhd loaded (

In [4]:
# Gathering Images in a dictionary
# The values represent the FVCs that were determined experimentally
Data = {'FLD1':0.223, 
        'FLD2':0.255, 
        'FLD3':0.286, 
        #'FLD10':0.179, #This scan caused unexplained abnormalities and is thus removed
        'FLD11':0.240, 
        'FLD12':0.266,
        'F1':0.230669,
        'F2':0.220811,
        'F3':0.230589,
        'CF1':0.255740,
        'CF2':0.223137,
        'CF3':0.228111,
        'C1':0.263567,
        'C2':0.231027,
        'C3':0.238066,
}

keys = [*Data.keys()] #Contains all Fibre Percentages
n0 = len(keys) #Initial amount of scans used

# Making array containing the 3D-arrays of all scans
Scans_float64 = np.array([globals()[keys[i]] for i in range(n0)])
print("64 Bit array size: ", getsizeof(Scans_float64)/1000, " KB")

# Reducing all values to 16 bits (equivalent to the maximum original information content of each CT scan)
Scans_float16 = np.array([np.float16(Scans_float64[i]) for i in range(n0)])
print("16 Bit array size: ", getsizeof(Scans_float16)/1000, " KB", "\n")
Scans = Scans_float16 #For simplicity, the variable name is adjusted

# Augmentation
print("Data Shape before Augemntation:", Scans.shape)
#Defining vectors for the geometrical operations
direct_x = 0
direct_y = 1
direct_z = 2
normalPlane_x = (1,2)
normalPlane_y = (2,0)
normalPlane_z = (0,1)
#Expanding the Scan set with augmented versions
#Both size inputs are identical to create cuboid-shaped arrays to allow for more augmentation steps 
Scans = addRotations(Scans, normalPlane_x, n0, newImageResolution, newImageResolution) #Adding 90° x-rotated scans to "Scans" array
Scans = addRotations(Scans, normalPlane_y, n0, newImageResolution, newImageResolution) #Adding 90° y-rotated scans to "Scans" array
Scans = addRotations(Scans, normalPlane_z, n0, newImageResolution, newImageResolution) #Adding 90° z-rotated scans to "Scans" array
Scans = addFlips(Scans, direct_x, newImageResolution, newImageResolution) #Adding x-flipped scants to "Scans" array
Scans = addFlips(Scans, direct_y, newImageResolution, newImageResolution) #Adding y-flipped scants to "Scans" array
Scans = addFlips(Scans, direct_z, newImageResolution, newImageResolution) #Adding z-flipped scants to "Scans" array
print("Data Shape after Augemntation:", Scans.shape, "\n")

# Creating the fiber precentages list
FibrePerc = np.array([*Data.values()])
fvc_appended = FibrePerc
for i in range(int(Scans.shape[0] / n0) - 1): # Mulitplying the Fibre Precentage list
    fvc_appended = np.append(fvc_appended, FibrePerc)
FibrePerc = fvc_appended
print("The FibrePerc list was multiplied", Scans.shape[0] / n0, "times", "\n")

# Defining the Split between input and validation Data
split_border = 2/3
split = round(len(Scans) * split_border) #Using 1/3 of Data for validation
print("split: ", split, "/", len(Scans)-split)

inputScans = Scans[:split] #Split for training data
InputScans = tf.expand_dims(inputScans, axis = 4)
inputLabels = FibrePerc[:split]

testScans = Scans[split:] #Split for validation data
TestScans = tf.expand_dims(testScans, axis = 4)
testLabels = FibrePerc[split:]

64 Bit array size:  112000.16  KB
16 Bit array size:  28000.16  KB 

Data Shape before Augemntation: (14, 100, 100, 100)
(28, 100, 100, 100)
(42, 100, 100, 100)
(56, 100, 100, 100)
(112, 100, 100, 100)
(224, 100, 100, 100)
(448, 100, 100, 100)
Data Shape after Augemntation: (448, 100, 100, 100) 

The FibrePerc list was multiplied 32.0 times 

split:  299 / 149


In [33]:
# Final CNN

def get_model(depth, width, height):
    inputs = keras.Input((depth, width, height, 1))
    
    x = tf.keras.layers.Normalization(axis=-1, invert=True)

    x = layers.Conv3D(filters=2, kernel_size=3, activation="relu")(inputs)
    x = layers.MaxPooling3D(pool_size=2)(x)
    
    x = layers.Dense(units=64, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    
    x = layers.Dense(units=128, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    
    x = layers.GlobalAveragePooling3D()(x)
    
    outputs = layers.Dense(units=1, activation="sigmoid")(x)
    return keras.Model(inputs, outputs, name="3dcnn")

In [34]:
#Training and prediction

#Defining adjustable learning rate
initial_learning_rate = 0.0001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

model = get_model(depth=newImageResolution, width=newImageResolution, height=newImageResolution) #Selecting the network architecture
model.summary() #prints out the overview over the network structure
model.compile(
    loss="binary_crossentropy"
    ,optimizer=keras.optimizers.Adam(learning_rate=lr_schedule)
    ,metrics=['mae']
)

# Initial prediction (without training)
y_m1 = model.predict(InputScans)
y_m1 = np.around(y_m1, 3).flatten() #reshape data for better visualisaition
deviation(inputLabels, y_m1)

# Train model
epochs = 40
hist = model.fit(Scans, FibrePerc, epochs = epochs, validation_split = 1 - split_border, shuffle = True)

# New prediction (after training)
y_m2 = model.predict(InputScans)
y_m2 = np.around(y_m2, 3).flatten() #reshape data for better visualisaition
deviation(inputLabels, y_m2)

# Predicting validation data
y_m3 = model.predict(TestScans)
y_m3 = np.around(y_m3, 3).flatten() #reshape data for better visualisaition
print("Result from test data prediction:")
deviation(testLabels, y_m3)

Model: "3dcnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 100, 100, 100, 1  0         
                             )]                                  
                                                                 
 conv3d_5 (Conv3D)           (None, 98, 98, 98, 2)     56        
                                                                 
 max_pooling3d_5 (MaxPooling  (None, 49, 49, 49, 2)    0         
 3D)                                                             
                                                                 
 dense_19 (Dense)            (None, 49, 49, 49, 64)    192       
                                                                 
 dropout_9 (Dropout)         (None, 49, 49, 49, 64)    0         
                                                                 
 dense_20 (Dense)            (None, 49, 49, 49, 128)   8320  

In [32]:
# Loss Plot
"""
import matplotlib
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rcParams.update({'font.size': 25})

fig = plt.figure(figsize=(10, 6), dpi=250)
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig('loss_normal_network_without_normalization.eps', format='eps')
plt.savefig('loss_normal_network_without_normalization.png', dpi=250, bbox_inches="tight")
#plt.show()

#Plotting results for the validation Data
X_test = np.arange(len(Scans)-split)
fig = plt.figure(figsize=(10, 6), dpi=250)
ax = fig.add_axes([0,0,1,1])
ax.bar(X_test + 0.00, testLabels, color = 'tab:red', width = 0.15)
ax.bar(X_test + 0.25, y_m3, color = 'green', width = 0.15)
ax.set_ylabel('Vol. fiber percentage')
ax.set_xlabel('testScans')
ax.legend(labels=['testLabels', 'prediction'])
plt.savefig('results_normal_network_without_normalization.eps', format='eps')
plt.savefig('results_normal_network_without_normalization.png', dpi=250, bbox_inches="tight")
plt.show()

# Saving values as .csv file to plot figure 30
np.savetxt("results_normal_network_deviation_without_normalization.csv", np.column_stack((testLabels, y_m3)), delimiter=",", header="original, predicted")

"""


'\nimport matplotlib\nplt.rc(\'text\', usetex=True)\nplt.rc(\'font\', family=\'serif\')\nplt.rcParams.update({\'font.size\': 25})\n\nfig = plt.figure(figsize=(10, 6), dpi=250)\nplt.plot(hist.history[\'loss\'])\nplt.plot(hist.history[\'val_loss\'])\n\nplt.title(\'model loss\')\nplt.ylabel(\'loss\')\nplt.xlabel(\'epoch\')\nplt.legend([\'train\', \'val\'], loc=\'upper left\')\nplt.savefig(\'loss_normal_network_without_normalization.eps\', format=\'eps\')\nplt.savefig(\'loss_normal_network_without_normalization.png\', dpi=250, bbox_inches="tight")\n#plt.show()\n\n#Plotting results for the validation Data\nX_test = np.arange(len(Scans)-split)\nfig = plt.figure(figsize=(10, 6), dpi=250)\nax = fig.add_axes([0,0,1,1])\nax.bar(X_test + 0.00, testLabels, color = \'tab:red\', width = 0.15)\nax.bar(X_test + 0.25, y_m3, color = \'green\', width = 0.15)\nax.set_ylabel(\'Vol. fiber percentage\')\nax.set_xlabel(\'testScans\')\nax.legend(labels=[\'testLabels\', \'prediction\'])\nplt.savefig(\'results_n

In [14]:
# Optional parameter sweep to optimize the network

""" 
def get_model_parametrized(depth, width, height, n_filters, n_layers, dropout, kernel, pool):
    inputs = keras.Input((depth, width, height, 1))

    x = tf.keras.layers.Normalization(axis=-1, invert=True)

    x = layers.Conv3D(filters=n_filters, kernel_size=kernel, activation="relu")(inputs)
    x = layers.MaxPooling3D(pool_size=pool)(x)
    
    for layer in range(1, n_layers+1):
        x = layers.Dense(units=8*2**layer, activation="relu")(x)
        x = layers.Dropout(dropout)(x)
    
    x = layers.GlobalAveragePooling3D()(x)
    
    outputs = layers.Dense(units=1, activation="sigmoid")(x)
    return keras.Model(inputs, outputs, name="3dcnn")


# Defining which and how many veriables are used for the sweep
# For actual sweep executions, a smaller set was used to avoid having to do 1500 iterations in one run
filters_list = [2, 4, 8, 16, 32]
layers_list = [1, 2, 3, 4, 5]
dropout_list = [0, 0.1, 0.3, 0.5, 0.8]
kernel_list = [1, 3, 5, 7]
pool_list = [2, 3, 4]

n_Iterations = len(filters_list)*len(layers_list)*len(dropout_list)*len(kernel_list)*len(pool_list)
results = np.zeros((n_Iterations, 8))

sweepCounter = 0
for n_filters in filters_list:
    for n_layers in layers_list:
        for dropout in dropout_list:
            for kernel in kernel_list:
                for pool in pool_list:
                    print("n_filters: ", n_filters, "| ", "n_layers:  ", n_layers, "| ", "dropout:  ", dropout, "| ", "kernel:  ", kernel, "| ", "pool:  ", pool)
                    print("Iteration: ", sweepCounter+1, "/", n_Iterations)

                    # Training Setup

                    initial_learning_rate = 0.0001
                    lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True) # Defining adjustable learning rate
    
                    model = get_model_parametrized(depth=newImageResolution, width=newImageResolution, height=newImageResolution, n_filters=n_filters, n_layers = n_layers, dropout=dropout, kernel=kernel, pool=pool)
                    model.compile(
                        loss="binary_crossentropy"
                        ,optimizer=keras.optimizers.Adam(learning_rate=lr_schedule)
                        ,metrics=['mae']
                    )

                    # Initial Prediction (without training)
                    y_m1 = model.predict(InputScans)

                    # Train Model
                    epochs = 40
                    hist = model.fit(Scans, FibrePerc, epochs = epochs, validation_split = 1 - split_border, verbose=0)

                    # New Prediction (after training)
                    y_m2 = model.predict(InputScans)
                    
                    # Predicting validation data
                    y_m3 = model.predict(TestScans)
                    print("Result from test data prediction:")
                    deviation(testLabels, y_m3)

                    #Saving Data
                    results[sweepCounter][0] = getSAD()
                    results[sweepCounter][1] = getSAD_rel()
                    results[sweepCounter][2] = n_filters
                    results[sweepCounter][3] = n_layers
                    results[sweepCounter][4] = dropout
                    results[sweepCounter][5] = kernel
                    results[sweepCounter][6] = pool
                    sweepCounter += 1

np.savetxt("results.csv", results, delimiter=",", header="SAD, SAD relative, n_filters, n_layers, dropout, n_kernels, pool")
"""

' \ndef get_model_parametrized(depth, width, height, n_filters, n_layers, dropout, kernel, pool):\n    inputs = keras.Input((depth, width, height, 1))\n\n    x = layers.Conv3D(filters=n_filters, kernel_size=kernel, activation="relu")(inputs)\n    x = layers.MaxPooling3D(pool_size=pool)(x)\n    \n    for layer in range(1, n_layers+1):\n        x = layers.Dense(units=8*2**layer, activation="relu")(x)\n        x = layers.Dropout(dropout)(x)\n    \n    x = layers.GlobalAveragePooling3D()(x)\n    \n    outputs = layers.Dense(units=1, activation="sigmoid")(x)\n    return keras.Model(inputs, outputs, name="3dcnn")\n\n\n# Defining which and how many veriables are used for the sweep\n# For actual sweep executions, a smaller set was used to avoid having to do 1500 iterations in one run\nfilters_list = [2, 4, 8, 16, 32]\nlayers_list = [1, 2, 3, 4, 5]\ndropout_list = [0, 0.1, 0.3, 0.5, 0.8]\nkernel_list = [1, 3, 5, 7]\npool_list = [2, 3, 4]\n\nn_Iterations = len(filters_list)*len(layers_list)*len(